# 4.2 Review Polarity

In [97]:
import nltk
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import ComplementNB, MultinomialNB

In [98]:
nltk.download('stopwords')
from nltk.corpus import stopwords
languages = [
    'arabic', 'azerbaijani', 'bengali', 'danish', 'dutch', 'english',
    'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian',
    'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian',
    'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish'
]
english_stopwords = set(stopwords.words('english'))
all_stopwords = set()
for language in languages:
    all_stopwords |= set(stopwords.words(language))

[nltk_data] Downloading package stopwords to C:\Users\Conor Mac
[nltk_data]     Amhlaoibh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [99]:
DIR_SAMPLE = '../data/samples/'
PATH_REVSENTS = DIR_SAMPLE + 'review_sentiment/%s.csv'

In [100]:
def preprocess_text(text, is_english=True):
    if is_english:
        text = text.lower()
        stopword_list = english_stopwords
    else:
        stopword_list = all_stopwords
    text = re.sub(r'[^\w\s]', ' ', text, re.UNICODE)
    text = ' '.join([
        word
        for word in text.split()
        if word not in stopword_list
    ])
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [101]:
def read_data(sample_name, train_size=0.8, seed=None):
    df = pd.read_csv(PATH_REVSENTS % sample_name)[['polarity', 'text']].reset_index(drop=True)
    if sample_name.startswith('eng'):
        df['text'] = df['text'].apply(lambda text: preprocess_text(text, is_english=True))
    else:
        df['text'] = df['text'].apply(lambda text: preprocess_text(text, is_english=False))
    train, validation = train_test_split(df, train_size=train_size, random_state=seed)
    return train, validation

In [102]:
train, validation = read_data('eng_eq_any_100000')
Xt, Yt = train['text'].tolist(), train['polarity'].tolist()
Xv, Yv = validation['text'].tolist(), validation['polarity'].tolist()

In [103]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), binary=True, smooth_idf=False)
Xt_tfidf = tfidf.fit_transform(Xt)
Xv_tfidf = tfidf.transform(Xv)

In [104]:
alphas = [0.1, 1, 10]
for alpha in alphas:
    model = MultinomialNB(alpha=alpha)
    auc = cross_val_score(model, Xt_tfidf, Yt, scoring='roc_auc', cv=5)
    print(auc.mean())

0.8783043859851218
0.8880166361078722
0.8764477140896604


In [105]:
model = MultinomialNB(alpha=1)
model.fit(Xt_tfidf, Yt)
probs = model.predict_proba(Xv_tfidf)
Yp = np.where(probs[:,1] >= 0.5, 1, 0).tolist()
print(accuracy_score(Yv, Yp))

0.80595


In [106]:
model = ComplementNB(alpha=1)
model.fit(Xt_tfidf, Yt)
probs = model.predict_proba(Xv_tfidf)
Yp = np.where(probs[:,1] >= 0.5, 1, 0).tolist()
print(accuracy_score(Yv, Yp))

0.80595


In [107]:
model = SGDClassifier(loss='hinge', penalty='l2', max_iter=20)
model.fit(Xt_tfidf, Yt)
Yp = model.predict(Xv_tfidf).tolist()
print(accuracy_score(Yv, Yp))

0.80205
